In [1]:
from tensorflow.keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras import Sequential
from data import Data

In [2]:

data = Data("data")
data.config_iter(64,3)
len(data)

4